In [1]:
import numpy as np
import random
import datetime
from tqdm import tqdm
import time

In [2]:
def compute_distance_km(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = lat1.astype('float'), lon1.astype('float'), lat2.astype('float'), lon2.astype('float')
    # 批量计算地球上两点间的球面距离
    R = 6371e3  # 地球半径（米）
    phi_1, phi_2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.power(np.sin(delta_phi / 2), 2) + np.cos(phi_1) * np.cos(phi_2) * np.power(np.sin(delta_lambda / 2), 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c / 1000.

In [3]:
class Ensemble_Model():
    def __init__(self, menbers):
        self.m = menbers
        self.w = np.ones([1, self.m]) / self.m

    def train_onetf(self, X, Y, dd=1, uu=1, vv=1, aa=1, rho=1):
        w0 = self.w / np.sum(self.w)
        u = np.zeros([1, self.m])
        for k, (x, y) in enumerate(zip(X, Y)):
            a = np.zeros([2 * self.m + 2, 2 * self.m + 2])
            b = np.zeros([2 * self.m + 2])
            t0, t1 = -y[0, 0], -y[0, 1] * (np.cos(np.radians(y[0, 1])) ** 2)
            for i in range(self.m):
                t0 += (w0[0, i] + u[0, i]) * x[i, 0]
                t1 += (w0[0, i] + u[0, i]) * x[i, 1] * (np.cos(np.radians(y[0, 1])) ** 2)

            for i in range(2 * self.m):
                for j in range(2 * self.m):
                    if i < self.m:
                        a[i, j] = 2 * dd * (x[j % self.m, 0] * x[i % self.m, 0]
                                            + x[j % self.m, 1] * (np.cos(np.radians(y[0, 1])) ** 2)
                                            * x[i % self.m, 1] * (np.cos(np.radians(y[0, 1])) ** 2))
                        if i == j:
                            a[i, j] = a[i, j] + 2 * uu
                    else:
                        a[i, j] = 2 * dd * (x[j % self.m, 0] * x[i % self.m, 0]
                                            + x[j % self.m, 1] * (np.cos(np.radians(y[0, 1])) ** 2)
                                            * x[i % self.m, 1] * (np.cos(np.radians(y[0, 1])) ** 2))
                        if i == j:
                            a[i, j] = a[i, j] + 2 * vv
                if i < self.m:
                    a[i, -2] = 1 * aa
                    b[i] = -2 * (dd * (t0 * x[i % self.m, 0] + t1 * x[i % self.m, 1] * (np.cos(np.radians(y[0, 1])) ** 2)) + u[0, i] * uu)
                else:
                    a[i, -1] = 1 * aa
                    b[i] = -2 * dd * (t0 * x[i % self.m, 0] + t1 * x[i % self.m, 1] * (np.cos(np.radians(y[0, 1])) ** 2))
            for j in range(self.m):
                a[-2, j] = 1 * aa
            for j in range(self.m, 2 * self.m):
                a[-1, j] = 1 * aa

            a = np.mat(a)
            b = np.mat(b).T
            c = np.linalg.solve(a, b)
            c = np.array(c.T)
            du = c[0, :self.m]
            u = u + du
        w0 = w0 + rho * u
        self.w = w0 * np.sum(self.w)

    def scale_onetf(self, X, Y, rho=0.1):
        for x, y in zip(X, Y):
            pred = self.predict(x)
            pred_, y_ = pred[0], y[0]
            scale = compute_scale(y_, pred_)
            self.w = self.w * (1 - (1 - scale) * rho)

    def get_w(self):
        return self.w

    def load_model(self, path):
        a = np.load(path, allow_pickle=True)
        self.w = a

    def predict(self, x):
        return np.matmul(self.w, x)

In [4]:
def Ensemble_train(epoch=1, model_path=None, num_menber=None):
    path = './pred/processed/'
    X = np.load(path + 'x_train.npy', allow_pickle=True) # ens成员的预测结果，shape=(n, m, 4, 2)，n个样本 * m个成员 * 4个时间步 * 经纬度
    if type(num_menber)==int: # 选择成员数
        X = X[:, :num_menber, :]
    Y = np.load(path + 'y_train.npy', allow_pickle=True).reshape(-1, 1, 4, 2) # 台风路径真实值，shape=(n, 4, 2)，为了后面循环取出1*2的向量，所以添加一个维度
    storm_num = np.load(path + 'train_tc_num.npy') # 所有台风的路径样本数按顺序排列的列表
    print('热带气旋总数:', len(storm_num)) # 台风数量
    time.sleep(0.01)
    # 以下变量和循环是为了将所有路径样本按不同台风分割储存
    start_index = 0
    x_storm = []
    y_storm = []
    for i in storm_num:
        end_index = start_index + i
        x_storm.append(X[start_index: end_index])
        y_storm.append(Y[start_index: end_index])
        start_index = end_index

    for time_step in range(4): # 4个时间步的集合预报模型分别训练
        print('训练' + str(int((time_step + 1) * 6)) + 'h提前期集合预报模型：')
        time.sleep(0.01)
        model = Ensemble_Model(X.shape[1]) # 将ens类实例化，确定ens成员数
        if model_path is not None: # 有训练好的模型想接着训练就按模型路径读入，实际上这个方法并不需要多次迭代和训练
            model.load_model(path)
        index = list(range(len(x_storm)))
        for i in range(epoch): # 打乱台风的顺序并投入模型训练
            start_time = datetime.datetime.now()
            random.shuffle(index)
            for j, k in tqdm(zip(index, list(range(len(storm_num))))):
                x = x_storm[j]
                y = y_storm[j]
                # model.train_onetf(x, y, 0.83239, 151.2719, 0.2699, 0.1235, 0.79983) OMuLeT的超参数
                model.train_onetf(x[:, :, time_step], y[:, :, time_step], 0.83239, 151.2719, 0.2699, 0.1235, 0.79983)
                np.save('./ens_model/model_' + str(X.shape[1]) + '_' + str(int((time_step + 1) * 6)) + 'h.npy', model.w)
            random.shuffle(index)
            end_time = datetime.datetime.now()
            print('epoch[' + str(i + 1) + '/' + str(epoch) + '] 耗时', (end_time - start_time).seconds, 's')
            time.sleep(0.01) # 为了让tqdm打印的更好看，虽然并没有发挥作用...

    # 以下是检验模型训练好的效果
    APE_all = np.zeros([2, 4])
    # 训练集
    for time_step in range(4):
        # 按预测时间步载入模型
        saved_model_path = './ens_model/model_' + str(X.shape[1]) + '_' + str(int((time_step + 1) * 6)) + 'h.npy'
        model = Ensemble_Model(X.shape[1])
        model.load_model(saved_model_path)
        pred = []
        for xx, yy in zip(X[:, :, time_step], Y[:, :, time_step]):
            p = model.predict(xx)
            pred.append([p[0, 0], p[0, 1], yy[0, 0], yy[0, 1]])
        pred_truth = np.array(pred)
        APE = compute_distance_km(pred_truth[:, 0], pred_truth[:, 1], pred_truth[:, 2], pred_truth[:, 3])
        print('APE ' + str(int((time_step + 1) * 6)) + 'h：', APE.mean(), 'SD:', np.std(APE))
        APE_all[0, time_step] = APE.mean()

    # 测试集
    X = np.load(path + 'x_test.npy', allow_pickle=True)
    if type(num_menber) == int:
        X = X[:, :num_menber, :]
    Y = np.load(path + 'y_test.npy', allow_pickle=True).reshape(-1, 1, 4, 2)
    for time_step in range(4):
        # 按预测时间步载入模型
        saved_model_path = './ens_model/model_' + str(X.shape[1]) + '_' + str(int((time_step + 1) * 6)) + 'h.npy'
        model = Ensemble_Model(X.shape[1])
        model.load_model(saved_model_path)
        pred = []
        for xx, yy in zip(X[:, :, time_step], Y[:, :, time_step]):
            p = model.predict(xx)
            pred.append([p[0, 0], p[0, 1], yy[0, 0], yy[0, 1]])
        pred_truth = np.array(pred)
        APE = compute_distance_km(pred_truth[:, 0], pred_truth[:, 1], pred_truth[:, 2], pred_truth[:, 3])
        print('APE ' + str(int((time_step + 1) * 6)) + 'h：', APE.mean(), 'SD:', np.std(APE))
        APE_all[1, time_step] = APE.mean()
    return APE_all

In [5]:
def Ensemble_test(model_path='./ens_model/', data_path='./pred/processed/', num_menber=None, is_save=False):
    APE_all = np.zeros([1, 4])
    # 测试集
    X = np.load(data_path + 'x_test.npy', allow_pickle=True)
    if type(num_menber) == int:
        X = X[:, :num_menber, :]
    Y = np.load(data_path + 'y_test.npy', allow_pickle=True).reshape(-1, 1, 4, 2)
    all_pred_truth = []
    for time_step in range(4):
        # 按预测时间步载入模型
        saved_model_path = model_path + 'model_' + str(X.shape[1]) + '_' + str(int((time_step + 1) * 6)) + 'h.npy'
        model = Ensemble_Model(X.shape[1])
        model.load_model(saved_model_path)
        pred = []
        for xx, yy in zip(X[:, :, time_step], Y[:, :, time_step]):
            p = model.predict(xx)
            pred.append([p[0, 0], p[0, 1], yy[0, 0], yy[0, 1]])
        pred_truth = np.array(pred)
        all_pred_truth.append(np.expand_dims(pred_truth, axis=1))
        APE = compute_distance_km(pred_truth[:, 0], pred_truth[:, 1], pred_truth[:, 2], pred_truth[:, 3])
        print('APE ' + str(int((time_step + 1) * 6)) + 'h：', APE.mean(), 'SD:', np.std(APE))
        APE_all[0, time_step] = APE.mean()
    all_pred_truth = np.array(all_pred_truth)
    if is_save:
        np.save('./pred/ens_test/model_' + str(X.shape[1]) + '_test_pred.npy', all_pred_truth)
    return APE_all

In [ ]:
num_menber = 5
Ensemble_train(1, num_menber=num_menber)
Ensemble_test(num_menber=num_menber)

In [ ]:
num_menber = 5
Ensemble_train(1, num_menber=num_menber)
Ensemble_test(num_menber=num_menber)